# Code to generate all data used for figures

## Imports

In [120]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import GroupShuffleSplit
from torch import nn
import json
import csv
from alphashape import alphashape
from shapely.geometry import Point

# Making no_avg_dataset.csv

This file is the processed embedding file. The processing takes the p value and run number from the file name and adds them as columns in the dataframe. It also removes the restart_box entries which are not used to train the model. 

In [10]:
# Functions to process the file
def extract_p(filename):
    p_junk = filename.split("_")[-1]
    p = float(p_junk.replace(".gsd", "").replace("p", ""))
    return p

def process_df(data):
    #Defining lists I use
    p_vals = []
    run_nos = []

    #Removing rows that are just the restart box
    for i, seq in enumerate(data['Sequence']):
        if len(seq)>21:
            data.drop(i, axis = 0, inplace = True)

    #Getting p values and run numbers from file names
    for file in data["Filename"]:
        p = extract_p(file)
        p_vals.append(p)

        run = float(file.split("_")[7])
        run_nos.append(run)
    
    #Adding p and run number to dataframe
    data['p'] = p_vals
    data['Run number'] = run_nos

    return data

In [11]:
# Start with the embedding file from UMAP
embedding_file = 'embeddings-8.csv'

# Load embeddings
df = pd.read_csv(embedding_file)

# Get processed file
processed_df = process_df(df)

# Save csv file
processed_df.to_csv('no_avg_dataset.csv')

# Making nn_input.npy, nn_output.npy, and nn_inputs_with_seq.csv

These files are the neural network training data. The input file contains the starting Z coordinate for a sequence and a p value, and the output file contains the real coordinate for that sequence at that p value. nn_inputs_wiht_seq.csv is the neural network input array with the corresponding sequence for each row. 

In [108]:
# Import processed embedding file 
df = pd.read_csv('no_avg_dataset.csv')

# Get unique sequences and p values
seqs = np.unique(df["Sequence"])
ps = np.unique(df["p"])

# Create empty arrays to store values in later
nn_input_array = np.empty((0,3))
nn_output_array = np.empty((0,2))
seq_array = np.empty((0,1))

# Loop through each sequence
for seq in seqs:
    seq_dataset = df[df["Sequence"] == seq]

    # Get the p=0 point
    p0_seq_dataset = seq_dataset[seq_dataset["p"] == 0]
    p0_z0 = p0_seq_dataset["Z0"]
    p0_z1 = p0_seq_dataset["Z1"]

    # Get the z coordinate at the other p values 
    for p in ps:
        p_seq_dataset = seq_dataset[seq_dataset["p"] == p]
        p_z0 = p_seq_dataset["Z0"]
        p_z1 = p_seq_dataset["Z1"]

        p_arr = np.full((5,1), p)

        # Put data into output arrays
        input_row = np.concatenate((np.asarray(p0_z0).reshape(-1,1), np.asarray(p0_z1).reshape(-1,1), p_arr), axis = 1)
        output_row = np.concatenate((np.asarray(p_z0).reshape(-1,1), np.asarray(p_z1).reshape(-1,1)), axis = 1)
        seq_row = np.asarray(p_seq_dataset["Sequence"]).reshape(-1,1)


        nn_input_array = np.vstack((nn_input_array, input_row))
        nn_output_array = np.vstack((nn_output_array, output_row))
        seq_array = np.vstack((seq_array, seq_row))

# Redefine arrays for simplicity
nia = nn_input_array
noa = nn_output_array

# Generate nia with sequence array 
nia_with_seq = np.concatenate((nia, seq_array), axis = 1)

In [111]:
# Write arrays to numpy files
np.save("nn_input.npy", nia)
np.save("nn_output.npy", noa)

# Write nia_with_seq to csv file 
np.savetxt("nn_inputs_with_seq.csv", nia_with_seq, delimiter = ',', fmt = "%s") # depending on formatting, fmt might not be required

# Save seq array for making the next csv file 
np.savetxt("seq_arr.npy", seq_array, fmt = "%s")

# Making error_fp_data.csv

In [37]:
# Import processed embedding file, nn_input, and nn_output
df = pd.read_csv('no_avg_dataset.csv')
nia = np.load("nn_input.npy")
noa = np.load("nn_output.npy")
seq_array = np.loadtxt("seq_arr.npy", dtype = str).reshape(-1,1)

# Get training, testing, and validation datasets, split by sequence
test_val_splitter = GroupShuffleSplit(n_splits = 1, test_size = 0.6, random_state = 0)

test_val_indices, train_indices = next(test_val_splitter.split(nia, noa, groups = seq_array))

x_test_val, x_train = nia[test_val_indices], nia[train_indices]
y_test_val, y_train = noa[test_val_indices], noa[train_indices]
groups_test_val = seq_array[test_val_indices]

val_test_splitter = GroupShuffleSplit(n_splits=1, test_size=0.5, random_state=0)
val_indices, test_indices = next(val_test_splitter.split(x_test_val, y_test_val, groups=groups_test_val))

x_val, x_test = x_test_val[val_indices], x_test_val[test_indices]
y_val, y_test = y_test_val[val_indices], y_test_val[test_indices]

test_seqs = seq_array[test_indices]
val_seqs = seq_array[val_indices]

#Turn them into tensors
x_train = torch.Tensor(x_train)
x_val = torch.Tensor(x_val)
y_train = torch.Tensor(y_train)
y_val = torch.Tensor(y_val)
x_test = torch.Tensor(x_test)
y_test = torch.Tensor(y_test)

In [40]:
# Define the model to get the predicted values

class MLPRegressor(nn.Module):
    def __init__(self, input_size, output_size, hidden_size=(100, ), activation=nn.ReLU()):
        torch.manual_seed(0)  # control random effects

        super(MLPRegressor, self).__init__()

        layers = []
        prev_layer_size = input_size
        for layer_size in hidden_size:
            layers.append(nn.Linear(prev_layer_size, layer_size))
            layers.append(activation)
            prev_layer_size = layer_size

        layers.append(nn.Linear(prev_layer_size, output_size))
        self.fc_layers = nn.Sequential(*layers)

    def forward(self, x):
        #x = x.clone().detach().requires_grad_(True)
        x = self.fc_layers(x)
        return x
    

def train_model(model, data, optimizer, n_epochs):
    torch.manual_seed(0)

    criterion = nn.MSELoss()
    x_train, x_val, y_train, y_val = data

    # do the training
    pbar = tqdm.tqdm(np.arange(n_epochs))
    for epoch in pbar:
        # Reset gradients
        optimizer.zero_grad()
        # Forward pass
        yp = model(x_train)
        # Compute Loss
        loss = criterion(yp, y_train)
        # Backward pass
        loss.backward()
        optimizer.step()
        # Update progress bar
        pbar.set_postfix_str(f'loss: {loss.item():.3e}')

    return model

# create a data structure to convert from str input to nn function
activation_dict = {'tanh': nn.Tanh(),
                   'relu': nn.ReLU(),
                   'leaky_relu': nn.LeakyReLU(),
                   'sigmoid': nn.Sigmoid(),
                   'elu': nn.ELU()}

def generate_model_and_optimizer(params):
    # create a list of layer sizes from the start, end, and depth
    raw_dims = np.linspace(params["layer_i"], params["layer_f"], params["num_layers"])
    hidden_size = tuple(raw_dims.round().astype(int))
    # choose the activation function
    activation = activation_dict[params['activation']]

    # initialize the model
    model = MLPRegressor(3, 2, hidden_size, activation)

    # initialize the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'])

    # return both objects to calling function
    return model, optimizer

def mlp_fitness(params):
    # the evaluation function cannot accept arguments -- we hard-code them here

    data = (x_train, x_val, y_train, y_val)
    try:
        # train on training set
        model, optimizer = generate_model_and_optimizer(params)
        train_model(model, data, optimizer, params['n_epochs'])
        # evaluate on held-out validation set
        y_pred = model(x_val).detach()
        rmse = torch.sqrt( torch.mean( (y_pred - y_val)**2 ) ).item()
    except:
        rmse = 1e12
    rmse = min(rmse, 1e2)  # need to choose a threshold RMSE to handle errors
    return np.log(rmse)    # log will be better behaved for very small numbers

In [42]:
# Load model 
with open('best_parameters_absolute.json', 'r') as f:
    best_parameters = json.load(f)

model, optimizer = generate_model_and_optimizer(best_parameters)

# Get predictedvalues
data = (x_train, x_val, y_train, y_val)
train_model(model, data, optimizer, best_parameters['n_epochs'])
# train rmse
y_pred_train = model(x_train).detach()
rmse_train = torch.sqrt( torch.mean( (y_pred_train - y_train)**2 ) ).item()
# validation rmse
y_pred_val = model(x_val).detach()
rmse_val = torch.sqrt( torch.mean( (y_pred_val - y_val)**2 ) ).item()
# test rmse
y_pred_test = model(x_test).detach()
rmse_test = torch.sqrt( torch.mean( (y_pred_test - y_test)**2 ) ).item()

100%|██████████| 1185/1185 [00:05<00:00, 205.50it/s, loss: 8.613e-01]


In [43]:
def euclidian_dist(prediction, expected):
    diff_squared = (prediction - expected) **2
    diff_sum = diff_squared[:,0] + diff_squared[:,1]
    dist = np.sqrt(diff_sum)

    return dist

train_dist = euclidian_dist(y_pred_train, y_train)
val_dist = euclidian_dist(y_pred_val, y_val)
test_dist = euclidian_dist(y_pred_test, y_test)

# Next, getting the original coordinates across the manifold

train_coord_dist = torch.cat((x_train, torch.unsqueeze(train_dist, dim = 1)), dim = 1)
val_coord_dist = torch.cat((x_val, torch.unsqueeze(val_dist, dim = 1)), dim = 1)
test_coord_dist = torch.cat((x_test, torch.unsqueeze(test_dist, dim = 1)), dim = 1)


# Train
unique_values_train = torch.unique(train_coord_dist[:, 2])
groups_train = {}

for value in unique_values_train:
    indices = torch.nonzero(train_coord_dist[:, 2] == value).squeeze()
    groups_train[value.item()] = train_coord_dist[indices]

# Val
unique_values_val = torch.unique(val_coord_dist[:, 2])
groups_val = {}

for value in unique_values_val:
    indices = torch.nonzero(val_coord_dist[:, 2] == value).squeeze()
    groups_val[value.item()] = val_coord_dist[indices]

# Test
unique_values_test = torch.unique(test_coord_dist[:, 2])
groups_test = {}

for value in unique_values_test:
    indices = torch.nonzero(test_coord_dist[:, 2] == value).squeeze()
    groups_test[value.item()] = test_coord_dist[indices]

In [74]:
# Next, want to get the average and standard deviation of the errors, like shift_from_average function

# Training first

train_keys = list(groups_train.keys())

avgs_train = []
stds_train = []

for group in train_keys:

    distances = np.array(groups_train[group])[:,3]
    avg_dist = np.average(distances)
    std = np.std(distances)

    avgs_train +=[avg_dist]
    stds_train += [std]


val_keys = list(groups_val.keys())

avgs_val = []
stds_val = []

for group in val_keys:

    distances = np.array(groups_val[group])[:,3]
    avg_dist = np.average(distances)
    std = np.std(distances)

    avgs_val +=[avg_dist]
    stds_val += [std]


test_keys = list(groups_test.keys())

avgs_test = []
stds_test = []

for group in test_keys:

    distances = np.array(groups_test[group])[:,3]
    avg_dist = np.average(distances)
    std = np.std(distances)

    avgs_test +=[avg_dist]
    stds_test += [std]

# Turning data into dictionary 

error_fp_data = {'avgs_train' : avgs_train,
                 'stds_train' :  stds_train,
                 'avgs_val' :  avgs_val, 
                 'stds_val' :  stds_val,
                 'avgs_test' :  avgs_test,
                 'stds_test' :  stds_test}


In [84]:
# Save to csv file
with open('error_fp_data.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in data.items():
        writer.writerow([key, value])

# Making error_for_datasets.pth

Requires the same model code as the previous file. Copied here for continuity.

In [112]:
# Import processed embedding file, nn_input, and nn_output
df = pd.read_csv('no_avg_dataset.csv')
nia = np.load("nn_input.npy")
noa = np.load("nn_output.npy")
seq_array = np.loadtxt("seq_arr.npy", dtype = str).reshape(-1,1)

# Get training, testing, and validation datasets, split by sequence
test_val_splitter = GroupShuffleSplit(n_splits = 1, test_size = 0.6, random_state = 0)

test_val_indices, train_indices = next(test_val_splitter.split(nia, noa, groups = seq_array))

x_test_val, x_train = nia[test_val_indices], nia[train_indices]
y_test_val, y_train = noa[test_val_indices], noa[train_indices]
groups_test_val = seq_array[test_val_indices]

val_test_splitter = GroupShuffleSplit(n_splits=1, test_size=0.5, random_state=0)
val_indices, test_indices = next(val_test_splitter.split(x_test_val, y_test_val, groups=groups_test_val))

x_val, x_test = x_test_val[val_indices], x_test_val[test_indices]
y_val, y_test = y_test_val[val_indices], y_test_val[test_indices]

test_seqs = seq_array[test_indices]
val_seqs = seq_array[val_indices]

#Turn them into tensors
x_train = torch.Tensor(x_train)
x_val = torch.Tensor(x_val)
y_train = torch.Tensor(y_train)
y_val = torch.Tensor(y_val)
x_test = torch.Tensor(x_test)
y_test = torch.Tensor(y_test)

In [113]:
# Define the model to get the predicted values

class MLPRegressor(nn.Module):
    def __init__(self, input_size, output_size, hidden_size=(100, ), activation=nn.ReLU()):
        torch.manual_seed(0)  # control random effects

        super(MLPRegressor, self).__init__()

        layers = []
        prev_layer_size = input_size
        for layer_size in hidden_size:
            layers.append(nn.Linear(prev_layer_size, layer_size))
            layers.append(activation)
            prev_layer_size = layer_size

        layers.append(nn.Linear(prev_layer_size, output_size))
        self.fc_layers = nn.Sequential(*layers)

    def forward(self, x):
        #x = x.clone().detach().requires_grad_(True)
        x = self.fc_layers(x)
        return x
    

def train_model(model, data, optimizer, n_epochs):
    torch.manual_seed(0)

    criterion = nn.MSELoss()
    x_train, x_val, y_train, y_val = data

    # do the training
    pbar = tqdm.tqdm(np.arange(n_epochs))
    for epoch in pbar:
        # Reset gradients
        optimizer.zero_grad()
        # Forward pass
        yp = model(x_train)
        # Compute Loss
        loss = criterion(yp, y_train)
        # Backward pass
        loss.backward()
        optimizer.step()
        # Update progress bar
        pbar.set_postfix_str(f'loss: {loss.item():.3e}')

    return model

# create a data structure to convert from str input to nn function
activation_dict = {'tanh': nn.Tanh(),
                   'relu': nn.ReLU(),
                   'leaky_relu': nn.LeakyReLU(),
                   'sigmoid': nn.Sigmoid(),
                   'elu': nn.ELU()}

def generate_model_and_optimizer(params):
    # create a list of layer sizes from the start, end, and depth
    raw_dims = np.linspace(params["layer_i"], params["layer_f"], params["num_layers"])
    hidden_size = tuple(raw_dims.round().astype(int))
    # choose the activation function
    activation = activation_dict[params['activation']]

    # initialize the model
    model = MLPRegressor(3, 2, hidden_size, activation)

    # initialize the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'])

    # return both objects to calling function
    return model, optimizer

def mlp_fitness(params):
    # the evaluation function cannot accept arguments -- we hard-code them here

    data = (x_train, x_val, y_train, y_val)
    try:
        # train on training set
        model, optimizer = generate_model_and_optimizer(params)
        train_model(model, data, optimizer, params['n_epochs'])
        # evaluate on held-out validation set
        y_pred = model(x_val).detach()
        rmse = torch.sqrt( torch.mean( (y_pred - y_val)**2 ) ).item()
    except:
        rmse = 1e12
    rmse = min(rmse, 1e2)  # need to choose a threshold RMSE to handle errors
    return np.log(rmse)    # log will be better behaved for very small numbers

In [114]:
# Load model 
with open('best_parameters_absolute.json', 'r') as f:
    best_parameters = json.load(f)

model, optimizer = generate_model_and_optimizer(best_parameters)

# Get predictedvalues
data = (x_train, x_val, y_train, y_val)
train_model(model, data, optimizer, best_parameters['n_epochs'])
# train rmse
y_pred_train = model(x_train).detach()
rmse_train = torch.sqrt( torch.mean( (y_pred_train - y_train)**2 ) ).item()
# validation rmse
y_pred_val = model(x_val).detach()
rmse_val = torch.sqrt( torch.mean( (y_pred_val - y_val)**2 ) ).item()
# test rmse
y_pred_test = model(x_test).detach()
rmse_test = torch.sqrt( torch.mean( (y_pred_test - y_test)**2 ) ).item()

100%|██████████| 1185/1185 [00:05<00:00, 215.59it/s, loss: 8.613e-01]


In [115]:
def euclidian_dist(prediction, expected):
    diff_squared = (prediction - expected) **2
    diff_sum = diff_squared[:,0] + diff_squared[:,1]
    dist = np.sqrt(diff_sum)

    return dist

# Get distances for three datasets
train_dist = euclidian_dist(y_pred_train, y_train)
val_dist = euclidian_dist(y_pred_val, y_val)
test_dist = euclidian_dist(y_pred_test, y_test)

# Combine coordinates and sitance data
train_coord_dist = torch.cat((x_train, torch.unsqueeze(train_dist, dim = 1)), dim = 1)
val_coord_dist = torch.cat((x_val, torch.unsqueeze(val_dist, dim = 1)), dim = 1)
test_coord_dist = torch.cat((x_test, torch.unsqueeze(test_dist, dim = 1)), dim = 1)

# Training set
unique_values_train = torch.unique(train_coord_dist[:, 2])
groups_train = {}

# Sort by p
for value in unique_values_train:
    indices = torch.nonzero(train_coord_dist[:, 2] == value).squeeze()
    groups_train[value.item()] = train_coord_dist[indices]

# Validation set
unique_values_val = torch.unique(val_coord_dist[:, 2])
groups_val = {}

# Sort by p
for value in unique_values_val:
    indices = torch.nonzero(val_coord_dist[:, 2] == value).squeeze()
    groups_val[value.item()] = val_coord_dist[indices]

# Test set
unique_values_test = torch.unique(test_coord_dist[:, 2])
groups_test = {}

# Sort by p
for value in unique_values_test:
    indices = torch.nonzero(test_coord_dist[:, 2] == value).squeeze()
    groups_test[value.item()] = test_coord_dist[indices]

# Save to dictionary
dist_dict = {'train_coord_dist' : train_coord_dist,
             'val_coord_dist' : val_coord_dist, 
             'test_coord_dist' : test_coord_dist}

In [ ]:
# Writing file 
torch.save(dist_dict, 'error_for_datasets.pth')

# Making norm_data.json

In [119]:
# Import data
df = pd.read_csv('no_avg_dataset.csv')

# Load model 
with open('best_parameters_absolute.json', 'r') as f:
    best_parameters = json.load(f)

model, optimizer = generate_model_and_optimizer(best_parameters)
model_dict = torch.load('embedding_model_noavg_absolute.pth')

#Loading pre-saved weights to the model
model.load_state_dict(model_dict)

<All keys matched successfully>

In [121]:
# Get points on a grid across the manifold 

# Getting coordinates and p values from dataframe
ps = np.unique(df["p"])
z_array = np.concatenate((np.asarray(df["Z0"]).reshape(-1,1), np.asarray(df["Z1"]).reshape(-1,1)), axis = 1)

# Getting x and y bounds for mesh
max_x = max(z_array[:,0])
max_y = max(z_array[:,1])

min_x = min(z_array[:,0])
min_y = min(z_array[:,1])

x_range = np.linspace(min_x, max_x, 30)
y_range = np.linspace(min_y, max_y, 30)
X,Y = np.meshgrid(x_range, y_range)

mesh_point_set = []

#Making an array of all the points
for i, row in enumerate(X): 
    row_mesh = np.concatenate((row.reshape(-1,1), Y[i].reshape(-1,1)), axis = 1)
    mesh_point_set.append(row_mesh)

mesh_set = np.vstack(mesh_point_set)

# Making alpha shape
alpha_shape = alphashape(z_array, alpha = 1)

# Collecting mesh points that are inside the alpha shape
alpha_shape_mesh_points = []
for point in mesh_set:
    point_shape = Point(point)
    if alpha_shape.contains(point_shape) == True:
        alpha_shape_mesh_points.append(point)
alpha_shape_mesh = np.vstack(alpha_shape_mesh_points)

In [122]:
# Function to get the norm values for a selected p 

def get_plot_norm(p, model, alpha_shape_mesh):
    at_p = p  #What p value are we looking at

    zero_arr = np.full((alpha_shape_mesh.shape[0], 1), at_p)
    grid_pts = np.hstack((alpha_shape_mesh, zero_arr))

    row_and_norm = []

    for i, row in enumerate(grid_pts):
        point = torch.Tensor(row)
        point.requires_grad_(True)

        #Forward pass
        output = model(point)
        z0, z1 = output

        z0.backward(torch.tensor(1.0, dtype = torch.float), retain_graph = True)

        gradient_norm_z0 = torch.norm(point.grad)

        # Zero out gradients for z0 for next iteration
        point.grad.zero_()

        # Backward pass to compute gradients for z1
        z1.backward(torch.tensor(1.0, dtype=torch.float))

        # Access the gradient for z1
        gradient_norm_z1 = torch.norm(point.grad)

        point.grad.zero_()

        # Save values 
        x = point[0].detach().numpy()
        y = point[1].detach().numpy()
        norm_z0 = gradient_norm_z0.item()  
        norm_z1 = gradient_norm_z1.item()  

        row_and_norm.append(np.array((x, y, norm_z0, norm_z1)))

        
    row_and_norm_arr = np.vstack(row_and_norm)

    dxdp = row_and_norm_arr[:,2].reshape(-1,1)
    dydp = row_and_norm_arr[:,3].reshape(-1,1)

    # Get magnitude of the norm 
    magnitude = np.sqrt(dxdp**2 + dydp**2)

    return row_and_norm_arr, magnitude

In [124]:
# Getting the norm data

# Going through each p
ps = np.unique(df["p"])

# Getting norm data for each p and saving to a list 
norm_data = []
for p in ps:
    row_and_norm_arr, magnitude = get_plot_norm(p, model, alpha_shape_mesh)
    norm_row_data = {'p' : p,
                     'row_and_norm_arr' : row_and_norm_arr,
                     'magnitude' : magnitude}
    norm_data += [norm_row_data]

# Convert the arrays in the dictionary to lists for saving
serial_norm_data = [{k: v.tolist() for k, v in d.items()} for d in norm_data]

In [125]:
# Write file
with open('norm_data.json', 'w') as f:
    json.dump(serial_norm_data, f)